In [1]:
import pandas as pd
import numpy as np
from sklearn.cluster import MeanShift, estimate_bandwidth
from tqdm import tqdm
import geopy.distance
import numpy as np
import random
import folium
import seaborn as sns
import matplotlib.pyplot as plt

In [2]:
# 전처리된 데이터 불러오기
trans = pd.read_excel('/Users/nabong/Desktop/project_login/data/cleaned/transport_info.xlsx')
accomd = pd.read_excel('/Users/nabong/Desktop/project_login/data/cleaned/accommodation.xlsx')
sight = pd.read_excel('/Users/nabong/Desktop/project_login/data/cleaned/관광지위치.xlsx')

In [3]:
trans.info()
trans.head(2)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10901 entries, 0 to 10900
Data columns (total 7 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   node_id   10901 non-null  object 
 1   정류소명(역명)  10901 non-null  object 
 2   월평균승차수    10901 non-null  float64
 3   월평균하차수    10901 non-null  float64
 4   월평균승하차총계  10901 non-null  float64
 5   위도        10901 non-null  float64
 6   경도        10901 non-null  float64
dtypes: float64(5), object(2)
memory usage: 596.3+ KB


,node_id,정류소명(역명),월평균승차수,월평균하차수,월평균승하차총계,위도,경도
0,B100000001,종로2가사거리,14645.0,13676.0,28321.0,37.569765,126.987750
1,B100000002,창경궁.서울대학교병원,81002.0,70081.0,151083.0,37.579183,126.996566


In [4]:
accomd.info()
accomd.head(2)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1221 entries, 0 to 1220
Data columns (total 15 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   이름         1221 non-null   object 
 1   종류         1221 non-null   object 
 2   평점         1215 non-null   float64
 3   숙박가격       1221 non-null   object 
 4   도보시간       1221 non-null   object 
 5   인기테그       1221 non-null   object 
 6   별점 높은순 후기  1221 non-null   object 
 7   별점 낮은순 후기  1213 non-null   object 
 8   주소         1221 non-null   object 
 9   객실수        1203 non-null   object 
 10  총후기        1221 non-null   int64  
 11  대실가격       837 non-null    object 
 12  대실시간       815 non-null    object 
 13  위도         1221 non-null   float64
 14  경도         1221 non-null   float64
dtypes: float64(3), int64(1), object(11)
memory usage: 143.2+ KB


,이름,종류,평점,숙박가격,도보시간,인기테그,별점 높은순 후기,별점 낮은순 후기,주소,객실수,총후기,대실가격,대실시간,위도,경도
0,앰배서더 서울 풀만 호텔,프리미엄호텔,4.8,250000,동대입구역 1번출구 250m,"['주차가능', '야외수영장', '수영장', '와이파이', '사우나', '클럽라운지...","['시설에 비해 가격도 저렴합니당', '너무너무 깨끗하고 쾌적합니다.', '룸서비스...",[],서울특별시 중구 동호로 287,269,169,NaN,NaN,37.560599,127.002136
1,글래드 여의도,4성급,4.7,200640,국회의사당역 4번출구 도보 1분,"['주차가능', '조식운영', '와이파이', '객실금연', '레스토랑', '바', ...","['정말 좋아요 말이 필요 없어요!', '컨디션은 항상 너무 좋네요! 최저가로 저렴...","['데이유즈 쪼매난 글씨에 속아 숙박전환 추가 55천원내고 아주 식겁시키네요', '...",서울특별시 영등포구 의사당대로 16,319,4416,NaN,NaN,37.527409,126.917265


In [5]:
sight.info()
sight.head(2)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1736 entries, 0 to 1735
Data columns (total 9 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   경도          1736 non-null   float64
 1   위도          1736 non-null   float64
 2   중심시도명       1736 non-null   object 
 3   중심시군구명      1736 non-null   object 
 4   중심관광지명      1736 non-null   object 
 5   Unnamed: 5  1736 non-null   object 
 6   중심카테고리 명_대  1736 non-null   object 
 7   중심카테고리 명_중  1736 non-null   object 
 8   순위          1736 non-null   int64  
dtypes: float64(2), int64(1), object(6)
memory usage: 122.2+ KB


,경도,위도,중심시도명,중심시군구명,중심관광지명,Unnamed: 5,중심카테고리 명_대,중심카테고리 명_중,순위
0,127.0591,37.5120,서울특별시,강남구,코엑스,서울 강남구 영동대로 513-0,관광지,문화관광,1
1,127.0597,37.5087,서울특별시,강남구,현대백화점/무역센터점,서울 강남구 테헤란로 517-0,관광지,쇼핑,2


In [6]:
sight['중심카테고리 명_중'].value_counts()

문화관광     957
쇼핑       453
역사관광     154
레저스포츠     64
기타관광      55
자연관광      44
체험관광       9
Name: 중심카테고리 명_중, dtype: int64

In [7]:
accomd_gps_list = list(zip(accomd['위도'], accomd['경도']))
accomd_gps_list[:5]

[(37.5605989670195, 127.002135954676),
 (37.5274094452659, 126.917265167328),
 (37.5428436189268, 126.95133595616),
 (37.4757184581365, 126.981123762876),
 (37.5276397334571, 126.964349149383)]

In [8]:
trans_gps_list = list(zip(trans['위도'], trans['경도']))
trans_gps_list[:5]

[(37.5697651251, 126.9877498816),
 (37.5791830159, 126.9965660023),
 (37.5826711749, 126.9983401004),
 (37.5685792736, 126.9876130976),
 (37.586243, 127.001744)]

In [9]:
sight_gps_list = list(zip(sight['위도'], sight['경도']))
sight_gps_list[:5]

[(37.512, 127.0591),
 (37.5087, 127.0597),
 (37.5273, 127.0275),
 (37.5279, 127.0416),
 (37.5284, 127.04)]

In [10]:
# 숙박업소의 주변 경쟁숙박업소 수, 최단/최장/평균거리를 구해주는 함수
def get_accomd_neighbors(accomd_gps_list):
    accomd['경쟁업소_수'] = 0
    accomd['경쟁업소_최단거리'] = None
    accomd['경쟁업소_최장거리'] = None
    accomd['경쟁업소_평균거리'] = None
    
    neighbors_dict = {}
    for i in tqdm(range(len(accomd_gps_list))):
        neighbors = []
        dist_list = []

        key = accomd.loc[i, '이름']
        accomd_gps = accomd_gps_list[i]
        for j in range(len(accomd_gps_list)):
            if i == j:
                continue;
            neighbor_gps = accomd_gps_list[j]
            dist = geopy.distance.distance(accomd_gps, neighbor_gps).km
            if dist <= 1:
                neighbors.append(neighbor_gps)
                dist_list.append(dist)
            
            neighbors_dict[key] = neighbors
        
        if len(dist_list) > 0:
            accomd.loc[i, '경쟁업소_수'] = len(neighbors)
            accomd.loc[i, '경쟁업소_최단거리'] = np.min(dist_list).round(2)
            accomd.loc[i, '경쟁업소_최장거리'] = np.max(dist_list).round(2)
            accomd.loc[i, '경쟁업소_평균거리'] = np.mean(dist_list).round(2)
    
    return neighbors_dict

100%|███████████████████████████████████████| 1221/1221 [04:37<00:00,  4.39it/s]


In [34]:
# 숙박업소의 주변 버스정류장 수, 지하철역수, 버스정류장 최단/평균거리, 지하철역 최단/평균거리, 교통유동인구 월평균 합계를 구해주는 함수
def get_transport_neighbors(accomd_gps_list, trans_gps_list):
    accomd['버스정류장_수(1km내)'] = 0
    accomd['지하철역_수(1km내)'] = 0
    accomd['버스정류장_최단거리(1km내)'] = None
    accomd['버스정류장_평균거리(1km내)'] = None
    accomd['지하철역_최단거리(1km내)'] = None
    accomd['지하철역_평균거리(1km내)'] = None
    accomd['교통유동인구_월평균승차수(1km내)'] = 0
    accomd['교통유동인구_월평균하차수(1km내)'] = 0
    accomd['교통유동인구_월평균승하차총계(1km내)'] = 0
    neighbors_dict = {}
    
    i = 0
    for accomd_gps in tqdm(accomd_gps_list):
        neighbors = []
        neighbors_bus = []
        neighbors_sub = []
        dist_list_bus = []
        dist_list_sub = []
        neighbors_in_pop = []
        neighbors_out_pop = []
        neighbors_total_pop = []
        key = accomd.loc[i, '이름']
        
        j = 0
        for trans_gps in trans_gps_list:
            dist = geopy.distance.distance(accomd_gps, trans_gps).km
            if dist <= 1:
                neighbors.append(trans_gps)
                if trans.loc[j, 'node_id'][0] == 'B':
                    neighbors_bus.append(trans_gps)
                    dist_list_bus.append(dist)
                if trans.loc[j, 'node_id'][0] == 'S':
                    neighbors_sub.append(trans_gps)
                    dist_list_sub.append(dist)
                neighbors_in_pop.append(trans.loc[j, '월평균승차수'])
                neighbors_out_pop.append(trans.loc[j, '월평균하차수'])
                neighbors_total_pop.append(trans.loc[j, '월평균승하차총계'])
    
            neighbors_dict[key] = neighbors
            j += 1
        if (len(neighbors_bus) > 0) & (len(neighbors_sub) > 0):
            accomd.loc[i, '버스정류장_수(1km내)'] = len(neighbors_bus)
            accomd.loc[i, '지하철역_수(1km내)'] = len(neighbors_sub)
            accomd.loc[i, '버스정류장_최단거리(1km내)'] = np.min(dist_list_bus).round(2)
            accomd.loc[i, '버스정류장_평균거리(1km내)'] = np.mean(dist_list_bus).round(2)
            accomd.loc[i, '지하철역_최단거리(1km내)'] = np.min(dist_list_sub).round(2)
            accomd.loc[i, '지하철역_평균거리(1km내)'] = np.mean(dist_list_sub).round(2)
            accomd.loc[i, '교통유동인구_월평균승차수(1km내)'] = np.mean(neighbors_in_pop).round(2)
            accomd.loc[i, '교통유동인구_월평균하차수(1km내)'] = np.mean(neighbors_out_pop).round(2)
            accomd.loc[i, '교통유동인구_월평균승하차총계(1km내)'] = np.mean(neighbors_total_pop).round(2)
    
        i += 1
        
    return neighbors_dict

100%|███████████████████████████████████████| 1221/1221 [41:40<00:00,  2.05s/it]


In [29]:
# 숙박업소의 주변 관광지수, 쇼핑몰 수, 관광지와의 최소/평균거리, 쇼핑몰과의 최소/평균거리를 구해주는 함수
def get_sight_neighbors(accomd_gps_list, sight_gps_list):
    accomd['관광지_수(1km내)'] = 0
    accomd['쇼핑몰_수(1km내)'] = 0
    accomd['관광지_최단거리(1km내)'] = None
    accomd['관광지_평균거리(1km내)'] = None
    accomd['쇼핑몰_최단거리(1km내)'] = None
    accomd['쇼핑몰_평균거리(1km내)'] = None
    neighbors_dict = {}
    
    i = 0
    for accomd_gps in tqdm(accomd_gps_list):
        neighbors = []
        neighbors_tourspot = []
        neighbors_shopping = []
        dist_list_tourspot = []
        dist_list_shopping = []
        
        key = accomd.loc[i, '이름']
        
        j = 0
        for sight_gps in sight_gps_list:
            dist = geopy.distance.distance(accomd_gps, sight_gps).km
            if dist <= 1:
                neighbors.append(sight_gps)
                if sight.loc[j, '중심카테고리 명_중'] == '쇼핑':
                    neighbors_shopping.append(sight_gps)
                    dist_list_shopping.append(dist)
                else:
                    neighbors_tourspot.append(sight_gps)
                    dist_list_tourspot.append(dist)
                    
    
            neighbors_dict[key] = neighbors
            j += 1
        
        if (len(neighbors_tourspot) > 0) & (len(neighbors_shopping) > 0):
            accomd.loc[i, '관광지_수(1km내)'] = len(neighbors_tourspot)
            accomd.loc[i, '쇼핑몰_수(1km내)'] = len(neighbors_shopping)
            accomd.loc[i, '관광지_최단거리(1km내)'] = np.min(dist_list_tourspot).round(2)
            accomd.loc[i, '관광지_평균거리(1km내)'] = np.mean(dist_list_tourspot).round(2)
            accomd.loc[i, '쇼핑몰_최단거리(1km내)'] = np.min(dist_list_shopping).round(2)
            accomd.loc[i, '쇼핑몰_평균거리(1km내)'] = np.mean(dist_list_shopping).round(2)
        
        i += 1
        
    return neighbors_dict

100%|███████████████████████████████████████| 1221/1221 [06:37<00:00,  3.07it/s]


In [ ]:
accomd_neighbors = get_accomd_neighbors(accomd_gps_list)
trans_neighbors = get_transport_neighbors(accomd_gps_list, trans_gps_list)
sight_neighbors = get_sight_neighbors(accomd_gps_list, sight_gps_list)

In [32]:
accomd.info()
accomd[['관광지_수(1km내)', '쇼핑몰_수(1km내)', '관광지_최단거리(1km내)', '관광지_평균거리(1km내)', '쇼핑몰_최단거리(1km내)', '쇼핑몰_평균거리(1km내)']]

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1221 entries, 0 to 1220
Data columns (total 34 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   이름                     1221 non-null   object 
 1   종류                     1221 non-null   object 
 2   평점                     1215 non-null   float64
 3   숙박가격                   1221 non-null   object 
 4   도보시간                   1221 non-null   object 
 5   인기테그                   1221 non-null   object 
 6   별점 높은순 후기              1221 non-null   object 
 7   별점 낮은순 후기              1213 non-null   object 
 8   주소                     1221 non-null   object 
 9   객실수                    1203 non-null   object 
 10  총후기                    1221 non-null   int64  
 11  대실가격                   837 non-null    object 
 12  대실시간                   815 non-null    object 
 13  위도                     1221 non-null   float64
 14  경도                     1221 non-null   float64
 15  경쟁업소

,관광지_수(1km내),쇼핑몰_수(1km내),관광지_최단거리(1km내),관광지_평균거리(1km내),쇼핑몰_최단거리(1km내),쇼핑몰_평균거리(1km내)
0,6,6,0.61,0.82,0.57,0.79
1,9,2,0.19,0.63,0.75,0.87
2,3,3,0.7,0.79,0.18,0.23
3,9,3,0.16,0.71,0.49,0.81
4,15,7,0.13,0.5,0.14,0.43
...,...,...,...,...,...,...
1216,14,3,0.22,0.66,0.38,0.62
1217,24,8,0.04,0.58,0.28,0.48
1218,2,2,0.26,0.45,0.09,0.43
1219,23,4,0.09,0.6,0.09,0.7


In [43]:
accomd.sort_values('지하철역_수(1km내)')[['이름', '지하철역_수(1km내)']]

,이름,지하철역_수(1km내)
359,Chason M (체이슨 엠),0
1161,면목 새롬,0
1162,중랑 발리,0
1163,상봉(중랑) 주연모텔,0
100,호텔 인피니,0
...,...,...
203,종로 호텔케이월드,14
195,호텔 리안,14
255,종로 그리드인 호텔,14
301,로얄 호텔 서울,14


In [40]:
accomd[accomd['버스정류장_수(1km내)'] == 0].iloc[:, 13:]

,위도,경도,경쟁업소_수,경쟁업소_최단거리,경쟁업소_최장거리,경쟁업소_평균거리,버스정류장_수(1km내),지하철역_수(1km내),버스정류장_최단거리(1km내),버스정류장_평균거리(1km내),...,지하철역_평균거리(1km내),교통유동인구_월평균승차수(1km내),교통유동인구_월평균하차수(1km내),교통유동인구_월평균승하차총계(1km내),관광지_수(1km내),쇼핑몰_수(1km내),관광지_최단거리(1km내),관광지_평균거리(1km내),쇼핑몰_최단거리(1km내),쇼핑몰_평균거리(1km내)
9,37.582358,126.886699,0,None,None,None,0,0,None,None,...,None,0.0,0.0,0.0,0,0,None,None,None,None
18,37.497435,126.845972,11,0.13,0.99,0.5,0,0,None,None,...,None,0.0,0.0,0.0,3,3,0.34,0.66,0.24,0.61
22,37.547896,126.819507,2,0.05,0.08,0.07,0,0,None,None,...,None,0.0,0.0,0.0,0,0,None,None,None,None
23,37.561118,126.839474,5,0.22,0.62,0.39,0,0,None,None,...,None,0.0,0.0,0.0,7,2,0.43,0.73,0.16,0.57
29,37.664946,127.007686,2,0.58,0.63,0.6,0,0,None,None,...,None,0.0,0.0,0.0,0,0,None,None,None,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1195,37.663727,127.036896,5,0.17,0.95,0.64,0,0,None,None,...,None,0.0,0.0,0.0,9,5,0.4,0.62,0.21,0.68
1200,37.670533,127.043406,5,0.3,0.95,0.57,0,0,None,None,...,None,0.0,0.0,0.0,9,5,0.31,0.52,0.2,0.54
1201,37.674010,127.044004,4,0.39,0.88,0.66,0,0,None,None,...,None,0.0,0.0,0.0,10,3,0.16,0.63,0.32,0.51
1202,37.665141,127.036021,5,0.17,0.88,0.66,0,0,None,None,...,None,0.0,0.0,0.0,8,5,0.26,0.57,0.05,0.64


In [36]:
accomd.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1221 entries, 0 to 1220
Data columns (total 34 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   이름                     1221 non-null   object 
 1   종류                     1221 non-null   object 
 2   평점                     1215 non-null   float64
 3   숙박가격                   1221 non-null   object 
 4   도보시간                   1221 non-null   object 
 5   인기테그                   1221 non-null   object 
 6   별점 높은순 후기              1221 non-null   object 
 7   별점 낮은순 후기              1213 non-null   object 
 8   주소                     1221 non-null   object 
 9   객실수                    1203 non-null   object 
 10  총후기                    1221 non-null   int64  
 11  대실가격                   837 non-null    object 
 12  대실시간                   815 non-null    object 
 13  위도                     1221 non-null   float64
 14  경도                     1221 non-null   float64
 15  경쟁업소

In [35]:
accomd.to_excel('accomd.xlsx', index=False)

In [ ]:
# x좌표, y좌표를 바탕으로 KMeans 알고리즘 적용해 최적화된 중심위치를 찾기
# K-means clustering 알고리즘을 적용해 주변 정류소가 모여있는 25개의 주요 위치 선정(구별로 중심점이 있을 것이라는 가정 하에 구의 개수인 25를 선정함)
# features = stop_gps[['Y좌표', 'X좌표']]
# model = KMeans(n_clusters = 25, random_state=42)
# model.fit(features)
# centers = model.cluster_centers_

In [ ]:
# # 평균 이동(Mean Shift) - 데이터의 밀도가 가장 높은 곳으로 중심 이동시키는 방식으로 군집화
# # 중심에 소속된 데이터의 평균 거리를 중심으로 이동하는 k-means에 비해 데이터의 밀도가 높은 곳으로 이동하며 군집의 개수를 미리 정해주지 않아도 된다는 장점이 있음
# features = df[['위도', '경도']]
# # 해당 알고리즘은 bandwith(대역폭) 속성이 큰 영향을 미치나 따로 지정을 안해줘도 최적화된 대역폭을 찾아 적용함
# #   -> 실행 결과 1개의 클러스터만 생성되어 최적 대역폭을 확인 후 대역폭을 조정
# print('최적 대역폭: ', estimate_bandwidth(features))

In [ ]:
# # 이웃정류장 딕셔너리에서 이웃정류장 위치만 가져오기 
# center_neighbors = get_neighbors(cluster_center, stop_gps_list).values()

In [ ]:
# ms = MeanShift(bandwidth=0.03, cluster_all=False, n_jobs=1)
# ms.fit(features) # 약 3분소요
# cluster_center = ms.cluster_centers_
# cluster_center = [tuple(c) for c in cluster_center]
# print('centers: ', cluster_center)

In [ ]:
# # Mean Shift 알고리즘 추천 위치 12곳 시각화
# # 나중에 folium.plugin의 marker cluster를 이용해 주변 정류소 수를 표현할 수 있는지 알아보자
# map = folium.Map(location=[37.541, 126.986], zoom_start=12)

# i = 1 
# for center in cluster_center:
#     folium.Marker(center, tooltip=f'location{i}', icon=folium.Icon(color='lightred', icon='star')).add_to(map)
#     i += 1
    
# for neighbors in center_neighbors:
#     for neighbor in neighbors:
#         folium.Circle(radius=10, location=[neighbor[0], neighbor[1]], color='#967BDC', fill=True).add_to(map)
# map